<a href="https://colab.research.google.com/github/ask-1407/my_portfolio/blob/main/NLP_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://kagglenote.com/kaggle/simpletransformers-claassification/

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install simpletransformers
!pip install mojimoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 5.4 MB/s 
     |████████████████████████████████| 10.3 MB 14.7 MB/s 
     |████████████████████████████████| 1.3 MB 49.1 MB/s 
     |████████████████████████████████| 1.9 MB 34.9 MB/s 
     |████████████████████████████████| 451 kB 43.1 MB/s 
     |████████████████████████████████| 7.6 MB 26.8 MB/s 
     |████████████████████████████████| 43 kB 968 kB/s 
     |████████████████████████████████| 5.5 MB 39.3 MB/s 
     |████████████████████████████████| 182 kB 43.2 MB/s 
     |████████████████████████████████| 168 kB 32.5 MB/s 
     |████████████████████████████████| 182 kB 41.0 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 168 kB 45.9 MB/s 
     |████████████████████████████████| 166 kB 50.8 MB/s 
     |████████████████████████████████| 166 kB 59.8 MB/s 
     |█████████████████████████████

In [3]:
#ColabでMeCabをインストールする
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [4]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score
import mojimoji
import MeCab
import requests

In [5]:
###SlothLibのStop wordリストをImport
url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
r = requests.get(url)
tmp = r.text.split('\r\n')
stopwords = []
for i in range(len(tmp)):
    if len(tmp[i]) < 1:
        continue
    stopwords.append(tmp[i])
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', 'ハイ', '上', '中', '下', '字', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', '段', '

In [56]:
 tagger = MeCab.Tagger()
 
 def tokenize(text, stop_words):
     node = tagger.parseToNode(text)
     result = []
     while node:
         features = node.feature.split(',')
         
         if features[0] != 'BOS/EOS':
             token = features[6] if features[6] != '*' else node.surface
             if token not in stop_words:
                 result.append(token)
         
         node = node.next
     
     return result

In [10]:
PATH = '/content/drive/My Drive/Colab Notebooks/nlp_data/train.csv'

data = pd.read_csv(PATH)

drop_col = ['id','source']
data = data.drop(drop_col, axis=1)

#前処理
data['text'] = data['text'].replace('\n', '', regex=True)# 改行コードを消去

# data.text=data.text.apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))
data.text=data.text.apply(lambda x: x.lower())#大文字を小文字へ
data.text= data.text.apply(mojimoji.han_to_zen) # 半角を全角へ
data.text=data.text.apply(lambda x:x.strip())


data.replace( stopwords, '', regex=True) #改行コードを除去 
data.to_csv('/content/drive/My Drive/Colab Notebooks/nlp_data/hoge.csv')

In [11]:
train_df,eval_df=train_test_split(data,test_size=0.1,shuffle=0.3,random_state=0)
train_df, eval_df

(                                                   text  label
 464             一年戦争はもうボロボロ陸ガン自体は好きだけど、つじつま合わせは流石に無いと思う      0
 1270                     ×職歴が人の価値＆＃９８９８；＆＃６５０３８；職歴無しはカス      0
 1974              要領よくいかないとだめだろ俺なんて普段からお中元、お歳暮バンバン贈ってるぜ      0
 1853  たとえば、「消費税増税しない。議論もしない」といってたのに、増税法案を通しちゃったよね。その...      0
 2259                守備はええし怪我はないし足もあるからなハムで１番替えのきかない選手やろ      0
 ...                                                 ...    ...
 4931                    だって払わないとモンスターと戦う権利すらないんだぞ仕方ないだろ      0
 3264  そもそもコネ入学使えなくなったから制服で振り落としたろ！っていうアレなのに金のことばっか騒い...      0
 1653  せやで日本に暮らしてる在日韓国人・朝鮮人の数は全体の０．６％それに対して苗字左右対称の人の数...      0
 2607  大学は野郎共とバイクで遊んだりクラブやら行ったり女の子らと酒飲んだり海行ったりして楽しかった...      0
 2732   ｎｏｔｅって言うあんま知られてないｓｎｓで愚痴って拡散これ拡散したのは案外関係者かも知れんなあｗ      0
 
 [4730 rows x 2 columns],
                                                    text  label
 3035  他の事件でも呼び出しくらうようになるのはそんな気にしていないんだけど・・・そういう問題ではな...      0
 138               掻きまくってる掻きまくってたら変な汁が出て来てその汁で余計に痒くなる悪循環      0
 629        

In [ ]:
#学習用パラメータ
params = {
    "max_seq_length": 256,#シークエンスの最大長。この値が大きいと文章の表現力が高くなってより高い精度が期待できる。大きくしすぎるとオーバーフィッティングや学習時間の増大などが生じる可能性がある。
    "train_batch_size": 32,#訓練時のバッチサイズ
    "eval_batch_size": 32,
    "num_train_epochs": 10,
    "use_early_stopping": True,
    "learning_rate": 1e-5,
    "n_gpu": 4,
    "overwrite_output_dir":True
}
weight = 1/ pd.DataFrame((train_df.label).tolist()).reset_index().groupby(0).count().values#不均衡データに対処するためのパラメーター
weight_sum = weight.sum()
weight /= weight_sum


model = ClassificationModel('roberta', 'roberta-base', args=params, num_labels=2, use_cuda=False)
model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(eval_df)

y_pred, raw_outputs = model.predict(eval_df.text.tolist())
print(accuracy_score(y_pred, eval_df.label))

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:613: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4730 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/148 [00:00<?, ?it/s]

In [69]:
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/nlp_data/test.csv')
test_data = test_data.drop(test_data.columns[1], axis=1) # sourceを削除


test_data['text'] = test_data['text'].replace('\n', '', regex=True)# 改行コードを消去

# data.text=data.text.apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))
test_data.text=test_data.text.apply(lambda x: x.lower())
test_data.text=test_data.text.apply(lambda x:x.strip())

In [70]:
predictions, raw_output =model.predict(test_data['text'].values.tolist())


  0%|          | 0/3223 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

In [71]:
### 結果の出力
result = pd.Series(predictions)
result = pd.DataFrame(result,columns=['label'])
result = pd.concat([test_data['id'],result],axis=1)
result.to_csv('/content/drive/My Drive/Colab Notebooks/nlp_data/result_bert.csv')

In [ ]:
pred_df = pd.concat([result,pd.DataFrame(test_data['text'],columns=['text'])],axis=1)
# pred_df

In [82]:
pred_df.loc[pred_df['label'] == 1,['text','label']]

,text,label
17,だから朴チョンヒは満州国軍から日本の士官学校に留学したんだよ朴チョンヒは満州国軍籍日本人がア...,1
37,日本人が怒ってるんだよ。日本人は生まれながらの卑怯者より、裏切り者を嫌う。,1
54,お前ら朝鮮人はそういう揚げ足だけは得意だよなwwwwwwww自動変換で出ただけなんyが？え？...,1
59,て言うか宮古人が嫌われとるロマとかユダヤ、在日みたいな存在,1
60,普通の日本人ならこう考えて当たり前なのにそういうこと言うあたりお前は在日朝鮮人かさっさと消えてくれ,1
...,...,...
3115,在特会はここでさっさと在日チョンが立ちのくまでしつこくデモしろよ,1
3127,人の権利だからね知的障害者は人じゃないよｗ,1
3171,このクソ食い民族めが！(*゜∀゜)ノ二足歩行するハエと変わらんのやでチョンはww,1
3193,本名じゃなかったらチョン確定じゃん(´･ω･`),1
